In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale
import matplotlib.pyplot as plt
from ktvgl_functions2 import init_parameters, fit_multivariate_t, L_update, w_update,  u_update, a_update, V_update, dual_update, residual_Update, update_parameters, updateADMMpenalties, L_star,L_from_w,D_star,D_from_w

In [2]:
log_rw_returns = pd.read_csv("log_rw_returns.csv")
log_returns = pd.read_csv("log_returns.csv")

In [3]:
log_returns.shape[1]

100

In [10]:


winLen = 200
r1 = min(100,log_returns.shape[1])
Xraw = log_returns.iloc[0:winLen, 0:r1] 
#r2 = min(5,log_rw_returns.shape[1])
#Xraw = log_rw_returns.iloc[0:winLen, 0:r2] 

maxIter = 10
update_rho = False
update_eta = True
early_stopping = False

#plt.plot(Xraw)

X = scale(Xraw.to_numpy())

#print(X[0:6,:])


# number of nodes
p = X.shape[1] # n_features
k = 1 # n_clusters

# number of observations
T_n = X.shape[0]      # n_observations in Frame
sigma_e = np.exp(10) # Student T Distribution Standard Dev. 
alpha = 2/(T_n*sigma_e)
beta = 2*np.log(sigma_e)/T_n


rho = 2    # ADMM hyperparameter. 
gamma = 10 # hyperparameter that controls the sparsity of VAR coefficients
eta = 1e-8 # hyperparameter that controls the regularization to obtain a k-component graph
#tau = 2 # Not used
#mu = 2 # Not used


# BOOK KEEPING
# residual vectors
primal_lap_residual = np.array([], dtype=float)
primal_deg_residual = np.array([], dtype=float)
dual_residual       = np.array([], dtype=float)

lagrangian = np.array([], dtype=float)
rho_seq = np.array([], dtype=float)
eta_seq = np.array([], dtype=float)

elapsed_time = np.array([], dtype=float)


res = fit_multivariate_t(X)
w, Aw, a, w_lagged, L_n, Phi_n, u_n, mu_vec, d, z_n, V = init_parameters(X,k)
#print("w",w)
nu = res["nu"]
nu = 5.176196 
print("nu: ",nu)
LstarXsq = [None]*(T_n)
for t in range(T_n):
    x = X[t, :]
    LstarXsq[t] = L_star(np.outer(x, x))


for iter in range(0,maxIter):

    #print("iter: ",iter+1)
    wUpdate, LwUpdate, AwUpdate   = w_update( X, w, w_lagged, a, nu, LstarXsq, L_n, u_n, mu_vec, z_n, Phi_n, V, rho, d, eta, beta)            
    #print("wUpdate")
    #print(wUpdate)
    #print("LwUpdate")
    #print(LwUpdate)
    #print("AwUpdate")
    #print(AwUpdate)
    
    u_nUpdate = u_update( wUpdate, a, w_lagged, mu_vec, rho, alpha)
    # x_update()
    aUpdate =a_update( w_lagged, wUpdate, u_nUpdate, mu_vec, gamma, rho)
    vUpdate = V_update(wUpdate,k)
    L_nUpdate = L_update(wUpdate, Phi_n, rho, k)
    Phi_nUpdate, mu_vecUpdate, z_nUpdate, Phi_n_res, u_n_res, z_n_res = dual_update(rho, wUpdate, L_nUpdate, Phi_n, aUpdate, w_lagged, mu_vec, u_nUpdate, d, z_n)
    primal_lap_residual, primal_deg_residual, dual_residual = residual_Update(rho, L_n, L_nUpdate, Phi_n_res, z_n_res, primal_lap_residual, primal_deg_residual, dual_residual) 
    rho, eta, rho_seq, eta_seq, has_converged = updateADMMpenalties( update_rho, rho, rho_seq,L_n, k, wUpdate, update_eta, eta, eta_seq, early_stopping)    
    L_n, w, u_n, a, V, Phi_n, mu_vec, z_n = update_parameters( L_nUpdate, wUpdate, u_nUpdate, aUpdate, vUpdate, Phi_nUpdate, mu_vecUpdate, z_nUpdate)

    if has_converged:
        break



np.set_printoptions(threshold=np.inf)  # no truncation
print("fit w",w)

nu:  5.176196
fit w [ 0.          0.          0.          0.          0.0292808  -0.
 -0.         -0.         -0.         -0.          0.02627805 -0.
 -0.          0.02500877  0.          0.02753746  0.         -0.
  0.03004594  0.          0.01634375  0.02644564  0.03438305  0.02733104
 -0.          0.         -0.          0.0288339   0.          0.0281143
 -0.         -0.         -0.          0.02676969 -0.          0.
 -0.          0.02545594  0.02950531  0.03024194  0.02811535  0.01584113
  0.          0.02793296  0.          0.02511943 -0.          0.
  0.         -0.          0.02716087 -0.         -0.          0.02836419
 -0.         -0.          0.02922696 -0.          0.02540493  0.
  0.         -0.         -0.          0.          0.         -0.
 -0.         -0.          0.02529225 -0.          0.         -0.
 -0.         -0.          0.02489099  0.          0.          0.
 -0.          0.         -0.         -0.          0.02658544  0.
 -0.         -0.         -0.          0

array([[ 7.95003256e-01, -2.03621029e-01,  8.06764220e-01,
         7.73520528e-01,  6.80865346e-02],
       [-1.04892837e+00, -1.05580540e+00, -4.64114812e-01,
         5.69097694e-01, -1.64151542e+00],
       [ 9.53167686e-01,  1.35275003e+00, -2.88982372e-01,
         8.03044729e-01, -1.50205666e-01],
       [ 1.65305867e+00,  2.04089628e+00,  1.94659265e+00,
        -1.62976042e+00, -5.13701061e-01],
       [-1.32515236e+00, -2.91802151e+00, -1.17553576e+00,
        -1.06709855e+00, -1.66696767e+00],
       [-1.33822425e+00, -8.05170460e-01,  2.11218133e-01,
        -1.55025110e+00, -4.00346360e-01],
       [-9.10939726e-01, -5.91838860e-01, -1.15098197e+00,
        -3.31458620e-01,  1.76506506e+00],
       [ 1.74943744e+00,  1.26962645e+00, -7.77105801e-01,
        -1.61433479e-01,  7.76163812e-01],
       [ 8.47423368e-02, -8.40403412e-01,  6.90548488e-01,
         2.60311225e-01,  4.17747678e-01],
       [-1.85625428e+00, -3.09432102e+00, -2.24489871e+00,
        -2.89737144e+00

In [43]:
def compute_kappa_from_marginals(Xc: np.ndarray, eps: float = 1e-12) -> float:
    """
    NumPy version of the R function:

      compute_kappa_from_marginals <- function(Xc) {
        T <- nrow(Xc)
        ki <- (T+1)*(T-1)/((T-2)*(T-3)) * ((colSums(Xc^4)/T)/(colSums(Xc^2)/T)^2 - 3*(T-1)/(T+1))
        k <- mean(ki)
        k_improved <- (T-1)/((T-2)*(T-3)) * ((T+1)*k + 6)
        k_improved/3
      }

    Xc should be column-centered (each column mean ~ 0).
    """
    Xc = np.asarray(Xc, dtype=float)
    T, p = Xc.shape
    if T <= 3:
        raise ValueError("T must be > 3 for the unbiased kurtosis correction.")

    m4j = np.mean( np.pow(Xc,4), axis=0)
    m2j = np.mean( np.pow(Xc,2), axis=0)

    kj_est = m4j/np.pow(m2j,2) - 3

    # average across columns, then apply the second correction and divide by 3  
    Kj_est = (T - 1) / ((T - 2) * (T - 3)) * ((T + 1) * kj_est + 6.0)
    kappa_est = (Kj_est.mean())/3.0 
    return kappa_est 

Xcentered = X - np.mean(X,axis=0)
T, p = Xcentered.shape
kappa_est2 = compute_kappa_from_marginals(Xcentered)
kappa_est1 = (-2/(p+2))
kappa_est = np.max([kappa_est1, kappa_est2])

if kappa_est <= 0:
    nu = 1e6
else:
    nu = 2.0 / kappa_est + 4.0


nu


1000000.0

In [69]:


def compute_kappa_from_marginals_numpy(Xc, eps=1e-12):
    """
    NumPy port of your R function, returning k_improved/3.
    Xc must be column-centered.
    """
    Xc = np.asarray(Xc, float)
    T, p = Xc.shape
    if T <= 3:
        raise ValueError("Need T > 3 for unbiased kurtosis correction.")
    m2 = (Xc**2).mean(axis=0)
    m4 = (Xc**4).mean(axis=0)
    m2 = np.maximum(m2, eps)
    factor1 = (T + 1) * (T - 1) / ((T - 2) * (T - 3))
    ki = factor1 * (m4 / (m2**2) - 3.0 * (T - 1) / (T + 1))  # per-column K_j
    k = ki.mean()
    k_improved = (T - 1) / ((T - 2) * (T - 3)) * ((T + 1) * k + 6.0)
    return k_improved / 3.0  # this matches your R routine's return value

def fit_nu_algorithm1(
    X,
    kappa_hat=None,
    max_outer=100,
    tol_rel=1e-2,          # 1% relative tolerance, as in the figure
    max_inner=50,          # inner fixed-point iterations for Sigma
    inner_tol=1e-6,
    jitter=1e-8,
    verbose=False,
):
    """
    Algorithm 1 (from your screenshot) implemented in NumPy.

    Parameters
    ----------
    X : (n, p) array
        Data (will be column-centered internally).
    kappa_hat : float or None
        If None, it is estimated from marginals (same as your R code).
    max_outer : int
        Max outer (nu) iterations.
    tol_rel : float
        Relative tolerance for nu (|nu_{t+1}-nu_t|/nu_t).
    max_inner : int
        Max inner iterations to solve the fixed-point covariance update.
    inner_tol : float
        Tolerance for convergence of Sigma in the inner loop.
    jitter : float
        Small ridge added to Sigma for numerical stability.
    verbose : bool
        Print progress.

    Returns
    -------
    nu : float
        Estimated degrees of freedom.
    Sigma : (p, p) array
        Covariance estimate from the last outer iteration.
    history : dict
        Diagnostics per outer iteration.
    """
    X = np.asarray(X, float)
    n, p = X.shape

    # center columns
    mu = X.mean(axis=0)
    Xc = X - mu

    # initialize nu from kappa, as per the figure
    if kappa_hat is None:
        #kappa_hat = compute_kappa_from_marginals_numpy(Xc)
        kappa_hat = compute_kappa_from_marginals(Xc)
    # guard: if kappa_hat <= 0, set a large nu (near Gaussian)
    if kappa_hat <= 0:
        nu = 1e6
    else:
        nu = 2.0 / kappa_hat + 4.0

    # initial covariance (regularized sample covariance)
    Sigma = (Xc.T @ Xc) / n
    Sigma += jitter * np.eye(p)

    hist = {"nu": [float(nu)], "theta": [], "inner_iters": []}

    for t in range(max_outer):
        # ---------- 1) Estimate Sigma for current nu via a fixed-point update ----------
        # We iterate: Sigma <- (1/n) * sum w_j * x_j x_j^T, with w_j = (p+nu)/(nu + delta_j)
        # where delta_j = x_j^T Sigma^{-1} x_j.
        inner = 0
        while inner < max_inner:
            inner += 1
            # precompute Sigma^{-1} and Mahalanobis distances
            try:
                Sigma_inv = np.linalg.inv(Sigma)
            except np.linalg.LinAlgError:
                # add ridge if nearly singular
                Sigma = Sigma + 10 * jitter * np.eye(p)
                Sigma_inv = np.linalg.inv(Sigma)

            # delta_j for all j
            # (Xc @ Sigma_inv) has shape (n, p); rowwise dot with Xc gives deltas
            AS = Xc @ Sigma_inv
            delta = np.einsum("ij,ij->i", AS, Xc)   # length n

            weights = (p + nu) / (nu + delta)       # length n

            # weighted covariance
            # (Xc^T * weights) @ Xc  / n
            Sigma_new = (Xc.T * weights) @ Xc / n
            Sigma_new = 0.5 * (Sigma_new + Sigma_new.T)  # enforce symmetry
            Sigma_new += jitter * np.eye(p)

            # check inner convergence
            rel_change = np.linalg.norm(Sigma_new - Sigma, ord="fro") / max(
                1e-12, np.linalg.norm(Sigma, ord="fro")
            )
            Sigma = Sigma_new
            if rel_change < inner_tol:
                break

        # ---------- 2) Update theta_t ----------
        # Use the (1 - p/n) correction shown in the figure with the current Sigma:
        Sigma_inv = np.linalg.inv(Sigma)
        AS = Xc @ Sigma_inv
        delta = np.einsum("ij,ij->i", AS, Xc)   # x_j^T Sigma^{-1} x_j
        theta = (1.0 - p / n) * (delta.mean())

        # ---------- 3) Update nu ----------
        nu_new = 2.0 * theta + 4.0

        hist["theta"].append(float(theta))
        hist["nu"].append(float(nu_new))
        hist["inner_iters"].append(inner)

        if verbose:
            print(f"[iter {t:02d}] nu={nu:.6g} -> {nu_new:.6g}  (theta={theta:.6g}, inner={inner})")

        # ---------- 4) Convergence check ----------
        if nu == 0:
            rel = np.inf
        else:
            rel = abs(nu_new - nu) / abs(nu)
        nu = nu_new
        if rel < tol_rel:
            break

    return float(nu), Sigma, hist




nu, Sigma, hist = fit_nu_algorithm1(
    Xcentered,
    kappa_hat=None,
    max_outer=100,
    tol_rel=1e-4,          # 1% relative tolerance, as in the figure
    max_inner=100,          # inner fixed-point iterations for Sigma
    inner_tol=1e-6,
    jitter=1e-8,
    verbose=False,
)
nu

14.78586541406342

In [ ]:


def loo_student_t_cov(X, i, nu_t, max_inner=50, tol=1e-6, jitter=1e-8):
    """
    Compute Σ̂_(i),t = leave-one-out covariance estimate for Student-t model.

    Parameters
    ----------
    X : array-like of shape (n, p)
        Data matrix (should be centered already).
    i : int
        Index of the observation to leave out.
    nu_t : float
        Current degrees of freedom parameter ν_t.
    max_inner : int
        Maximum fixed-point iterations.
    tol : float
        Convergence tolerance (relative Frobenius norm).
    jitter : float
        Diagonal regularization term to keep Σ̂ positive definite.

    Returns
    -------
    Sigma_i : (p, p) ndarray
        The leave-one-out covariance estimate.
    """
    X = np.asarray(X, float)
    n, p = X.shape

    # --- Leave-one-out step (this is where j != i happens!)
    X_loo = np.delete(X, i, axis=0)  # (n-1, p)

    # Initialize Σ̂_(i),t (use empirical covariance over leave-one-out data)
    Sigma_i = (X_loo.T @ X_loo) / n
    Sigma_i += jitter * np.eye(p)

    for _ in range(max_inner):
        # Compute Mahalanobis distances: δ_j = x_j^T Σ^{-1} x_j  (for j != i)
        A = np.linalg.solve(Sigma_i, X_loo.T).T      # (n-1, p)
        delta = np.einsum('ij,ij->i', A, X_loo)      # length (n-1)

        # Compute Student-t weights
        w = (p + nu_t) / (nu_t + delta)              # (n-1,)

        # Weighted covariance estimate
        Sigma_new = (X_loo.T * w) @ X_loo / n
        Sigma_new = 0.5 * (Sigma_new + Sigma_new.T)  # symmetrize
        Sigma_new += jitter * np.eye(p)

        # Check convergence
        rel_change = np.linalg.norm(Sigma_new - Sigma_i, 'fro') / np.linalg.norm(Sigma_i, 'fro')
        Sigma_i = Sigma_new
        if rel_change < tol:
            break

    return Sigma_i


14.782315146648713

In [141]:
import math
import numpy as np


def nu_mle_diag_resampled(
    X,
    *,
    fT_resampling: int = 4,
    N_resampling_factor: float = 1.2,
    nu_min: float = 2.5,
    nu_max: float = 100.0,
    random_state: int | None = None
):
    """
    NumPy translation of the R 'MLE-mv-diagcov-resampled' (aka 'MLE-diag-resampled') for nu,
    and returns (mu, Sigma(scatter), cov) consistent with the diagonal-covariance assumption.

    Returns:
        dict with keys:
            - 'nu'     : MLE of degrees of freedom
            - 'mu'     : sample mean vector (shape (N,))
            - 'Sigma'  : scatter matrix (shape (N,N)), Sigma = ((nu-2)/nu) * cov
            - 'cov'    : diagonal covariance matrix from sample variances (shape (N,N))
    """
    X = np.asarray(X, dtype=float)
    if X.ndim != 2:
        raise ValueError("X must be a 2D array (T x N).")
    T, N = X.shape
    if T <= N:
        raise ValueError("Need T > N for this estimator (after any NA removal).")
    if nu_min <= 2:
        raise ValueError("nu_min must be > 2.")
    if nu_max <= nu_min:
        raise ValueError("nu_max must be > nu_min.")

    rng = np.random.default_rng(random_state)

    # ---- mean / centered data ----
    mu = X.mean(axis=0)
    Xc = X - mu

    # ---- diagonal covariance (sample variances with T-1) ----
    var = (Xc**2).sum(axis=0) / (T - 1)
    if np.any(var <= 0):
        raise ValueError("Non-positive variance encountered in a column.")
    cov = np.diag(var)

    # ---- delta^2 per entry ----
    delta2_var = Xc**2 / var

    # ---- resampling trick to form row-wise 'diagonal Mahalanobis' sums ----
    Tf = T * fT_resampling
    delta_rep = np.tile(delta2_var, (fT_resampling, 1))  # (Tf, N)
    N_resampling = int(round(N_resampling_factor * N))
    idx = rng.integers(0, N, size=(Tf, N_resampling))
    delta2_cov = delta_rep[np.arange(Tf)[:, None], idx].sum(axis=1)  # (Tf,)

    # ---- negative log-likelihood in nu (exactly as in R code branch) ----
    def negLL(nu: float) -> float:
        if nu <= 2.0:
            return np.inf
        term1 = (N * Tf) * 0.5 * (math.log((nu - 2.0) / nu))
        scaled = nu + (nu / (nu - 2.0)) * delta2_cov
        if np.any(scaled <= 0):
            return np.inf
        term2 = ((N + nu) * 0.5) * np.log(scaled).sum()
        term3 = -Tf * math.lgamma((N + nu) * 0.5)
        term4 =  Tf * math.lgamma(nu * 0.5)
        term5 = -(nu * Tf * 0.5) * math.log(nu)
        return term1 + term2 + term3 + term4 + term5

    # ---- golden-section search on [nu_min, nu_max] ----
    def golden_section_minimize(f, a, b, tol=1e-5, max_iter=200):
        invphi = (math.sqrt(5) - 1) / 2
        invphi2 = (3 - math.sqrt(5)) / 2
        c = a + invphi2 * (b - a)
        d = a + invphi * (b - a)
        fc = f(c)
        fd = f(d)
        for _ in range(max_iter):
            if abs(b - a) < tol * (abs(a) + abs(b)):
                break
            if fc < fd:
                b, d, fd = d, c, fc
                c = a + invphi2 * (b - a)
                fc = f(c)
            else:
                a, c, fc = c, d, fd
                d = a + invphi * (b - a)
                fd = f(d)
        return c if fc < fd else d

    nu_hat = float(golden_section_minimize(negLL, nu_min, nu_max))

    # ---- scatter from covariance via cov = (nu/(nu-2)) * Sigma ----
    Sigma = ((nu_hat - 2.0) / nu_hat) * cov

    return {
        "nu": nu_hat,
        "mu": mu,
        "Sigma": Sigma,
        "cov": cov,
    }


# --- small demo ---
if __name__ == "__main__":
    # Generate a quick synthetic example (not heavy-tailed; just to show it runs)
    rng = np.random.default_rng(0)
    T, N = 1000, 5
    X_demo = rng.standard_normal((T, N))
    res = nu_mle_diag_resampled(Xcentered, random_state=None)
   

print(res["nu"])
print(res["mu"])
print(res["cov"])



9.088774465672078
[ 0.00000000e+00 -2.66453526e-17  4.44089210e-18  1.77635684e-17
 -1.77635684e-17]
[[1.00502513 0.         0.         0.         0.        ]
 [0.         1.00502513 0.         0.         0.        ]
 [0.         0.         1.00502513 0.         0.        ]
 [0.         0.         0.         1.00502513 0.        ]
 [0.         0.         0.         0.         1.00502513]]


In [142]:
from scipy.stats import multivariate_t
rv = multivariate_t([1.0, -0.5], [[2.1, 0.3], [0.3, 1.5]], df=2)

In [172]:
import math
import numpy as np

# --- keep your previously defined nu_mle_diag_resampled() here ---
# (or the version that also returns mu/cov/Σ; we'll reuse only the nu)

def nu_mle_diag_resampled(X, *, fT_resampling=4, N_resampling_factor=1.2,
                          nu_min=2.5, nu_max=100.0, random_state=None):
    X = np.asarray(X, dtype=float)
    T, N = X.shape
    rng = np.random.default_rng(random_state)
    mu = X.mean(axis=0)
    Xc = X - mu
    var = (Xc**2).sum(axis=0) / (T - 1)
    if np.any(var <= 0):
        raise ValueError("Non-positive variance encountered.")

    delta2_var = Xc**2 / var
    Tf = T * fT_resampling
    delta_rep = np.tile(delta2_var, (fT_resampling, 1))
    N_resampling = int(round(N_resampling_factor * N))
    idx = rng.integers(0, N, size=(Tf, N_resampling))
    delta2_cov = delta_rep[np.arange(Tf)[:, None], idx].sum(axis=1)

    def negLL(nu: float) -> float:
        if nu <= 2.0: return np.inf
        term1 = (N * Tf) * 0.5 * (math.log((nu - 2.0) / nu))
        scaled = nu + (nu / (nu - 2.0)) * delta2_cov
        if np.any(scaled <= 0): return np.inf
        term2 = ((N + nu) * 0.5) * np.log(scaled).sum()
        term3 = -Tf * math.lgamma((N + nu) * 0.5)
        term4 =  Tf * math.lgamma(nu * 0.5)
        term5 = -(nu * Tf * 0.5) * math.log(nu)
        return term1 + term2 + term3 + term4 + term5

    def golden_section_minimize(f, a, b, tol=1e-5, max_iter=200):
        invphi = (math.sqrt(5) - 1) / 2
        invphi2 = (3 - math.sqrt(5)) / 2
        c = a + invphi2 * (b - a); d = a + invphi * (b - a)
        fc, fd = f(c), f(d)
        for _ in range(max_iter):
            if abs(b - a) < tol * (abs(a) + abs(b)): break
            if fc < fd:
                b, d, fd = d, c, fc
                c = a + invphi2 * (b - a); fc = f(c)
            else:
                a, c, fc = c, d, fd
                d = a + invphi * (b - a); fd = f(d)
        return c if fc < fd else d

    return float(golden_section_minimize(negLL, nu_min, nu_max))

# --- EM fit of mu and full Sigma with fixed nu (no missing data, no factor model) ---
def fit_mvt_em_mu_sigma(
    X,
    nu: float | None = None,
    *,
    estimate_nu: bool = True,
    nu_min: float = 2.5,
    nu_max: float = 100.0,
    max_iter: int = 200,
    ptol: float = 1e-3,
    px_em: bool = True,
    jitter: float = 1e-9,
    random_state: int | None = None
):
    """
    EM updates for mu and full scatter Sigma of a multivariate t with fixed nu.
    If estimate_nu=True and nu is None, nu is obtained with diag-resampled MLE (one-shot).

    Returns:
        dict(mu, Sigma(scatter), cov, nu, converged, num_iterations)
    """
    X = np.asarray(X, dtype=float)
    if X.ndim != 2:
        raise ValueError("X must be 2D (T x N).")
    T, N = X.shape
    if T <= N:
        raise ValueError("Need T > N.")

    # 1) One-shot nu (if requested)
    if estimate_nu and (nu is None):
        nu = nu_mle_diag_resampled(X, nu_min=nu_min, nu_max=nu_max, random_state=random_state)
    if nu is None:
        raise ValueError("nu must be provided or estimate_nu=True.")

    # 2) Initialize mu, Sigma like in R (scatter from sample cov)
    mu = X.mean(axis=0)
    Xc = X - mu
    cov_sample = (Xc.T @ Xc) / (T - 1)                # sample covariance
    Sigma = ((max(nu, 2.1) - 2.0) / max(nu, 2.1)) * cov_sample  # scatter init

    # 3) EM loop
    def fnu(nu):  # used in R stopping crit
        return nu / (nu - 2.0)

    for it in range(1, max_iter + 1):
        mu_old = mu.copy()
        Sigma_old = Sigma.copy()

        # E-step
        # r2_t = x_c^T Sigma^{-1} x_c   for each row
        # add tiny jitter to ensure SPD in solve
        Sigma_j = Sigma + jitter * np.eye(N)
        Sinv = np.linalg.inv(Sigma_j)
        Xc = X - mu  # current centering
        r2 = np.einsum("ti,ij,tj->t", Xc, Sinv, Xc)   # shape (T,)
        E_tau = (N + nu) / (nu + r2)                  # shape (T,)

        ave_E_tau = E_tau.mean()
        ave_E_tau_X = (E_tau[:, None] * X).mean(axis=0)  # (1/T) * sum_t E_tau_t * x_t

        # M-step
        mu = ave_E_tau_X / ave_E_tau
        Xc = X - mu
        # (1/T) * Xc^T diag(E_tau) Xc
        Sigma = (Xc.T * E_tau) @ Xc / T
        if px_em:
            Sigma = Sigma / ave_E_tau  # PX-EM acceleration (alpha = ave_E_tau)

        # stopping: relative change of mu and Sigma
        mu_denom = np.maximum(1.0, np.abs(mu_old) + np.abs(mu))
        mu_conv = np.all(np.abs(mu - mu_old) <= 0.5 * ptol * mu_denom)

        Sig_denom = np.maximum(1.0, np.abs(Sigma_old) + np.abs(Sigma))
        Sig_conv = np.all(np.abs(Sigma - Sigma_old) <= 0.5 * ptol * Sig_denom)

        if mu_conv and Sig_conv:
            converged = True
            break
    else:
        converged = False
        it = max_iter

    # covariance from scatter
    cov = (nu / (nu - 2.0)) * Sigma if nu > 2 else np.full_like(Sigma, np.nan)

    return {
        "mu": mu,
        "Sigma": Sigma,   # scatter
        "cov": cov,       # covariance
        "nu": nu,
        "converged": converged,
        "num_iterations": it,
    }

res = fit_mvt_em_mu_sigma(X, estimate_nu=True, nu_min=2.5, nu_max=100.0)


print(res["nu"])
print("")
print( res["mu"])
print("")
print(res["cov"])

10.911119633206962

[0.01800573 0.02508158 0.01652527 0.02904574 0.02457702]

[[1.09113484 0.37627442 0.40995221 0.27064171 0.35586962]
 [0.37627442 1.05265193 0.39376667 0.36286855 0.34279891]
 [0.40995221 0.39376667 1.07510936 0.31869292 0.33349309]
 [0.27064171 0.36286855 0.31869292 1.08762413 0.26794586]
 [0.35586962 0.34279891 0.33349309 0.26794586 1.06854188]]
